A notebook for processing flat field data for each channel of OROS (the OROCHI Simulator).

# Process Overview

- load flat-field images
- average frames from different spectralon positions
- look for convergence toward mean pixel values
    - by plotting the mean change-in-mean across all pixels
    - i.e. change in mean is this abs(mean - last mean) - pixel wise
    - mean of this over all pixels shows how this changes for the repeat images.
- save final flat field